# Projet final

### Acquisition et chargement des données

* Récupération des fichiers Excel avec les classements
* Mise en place d'une copie locale des fichiers Excel afin de ne pas les recharger à chaque run.
* Vers la fin de la course le format des fichiers Excel change avec les arrivées des voiliers : il est possible de s'arrêter juste avant.
* Extraction des caractéristiques techniques de chacun des voiliers.




In [1]:
# Chargement des libs
import pandas as pd
import os
import sys
from bs4 import BeautifulSoup as bs
import requests
import datetime as dt
import dateparser
import numpy as np
import matplotlib.pyplot as plt


# libs for xlsx "xxid" fix
import tempfile
from zipfile import ZipFile
import shutil
from fnmatch import fnmatch
import re
import glob
#

# variables utiliséees globalement
URL_RESULTS="https://www.vendeeglobe.org/fr/classement"
FILE_RESULTS="classement.html"
URL_GLOSSAIRE="https://www.vendeeglobe.org/fr/glossaire"
FILE_GLOSSAIRE="glossaire.html"
EXCELS_DIR="results"
PICKLE_DF="pickle_classement.pkl"
PICKLE_DF_TECH="pickle_classement_et_tech.pkl"


* Récupération des fichiers Excel avec les classements
* Mise en place d'une copie locale des fichiers Excel afin de ne pas les recharger à chaque run.
* Chargement dans un dataframe (clean up et split des noms de colonne, préparation des data pour traitements)

In [2]:
def get_soup_from_url(url):
    """
    Retourne la soupe de l'url du fichier html passé en paramètre
    """
    res = requests.get(url)
    soup = bs(res.content, 'html.parser')
    return soup

def get_soup_from_file(file):
    """
    Retourne la soupe du fichier html passé en paramètre
    """
    soup = bs(file, 'html.parser')
    return soup

def parse_url_for_excels(url):
    """
    Récupère la liste des fichiers excel et les télécharge dans le répertoire "results/" (EXCELS_DIR)
    """
    print("Getting url of files to download...")
    soup = get_soup_from_url(url)
    dates_list = []
    for option in soup.find_all('option'):
        if option['value'] != '':
            dates_list.append(option['value'])
    
    # format de fichiers à récupérer 
    # https://www.vendeeglobe.org/download-race-data/vendeeglobe_20210305_080000.xlsx
    print("Downloading xlsx files...")
    for date in dates_list:
        xlsx_name = f'vendeeglobe_{date}.xlsx'
        xlsx_file = requests.get(f"https://www.vendeeglobe.org//download-race-data/{xlsx_name}")
        open(os.path.join(EXCELS_DIR, xlsx_name), 'wb').write(xlsx_file.content)

def fix_xlsx_errors():
    """
    Fix des fichiers xlsx, un header xxid dans un des fichiers du xlsx n'est pas reconnu par openpyxl
    """
    print("Fixing xlsx files...")
    for file in [f for f in glob.glob(EXCELS_DIR + "/*.xlsx")]:
        change_in_zip(file, name_filter='xl/styles.xml', # the problematic property is found in the style xml files
                      change=lambda d: re.sub(b'xxid="\d*"', b"", d))
        
# fix of xlsx files
def change_in_zip(file_name, name_filter, change):
    """
    le fix appliqué à chaque fichier
    """
    tempdir = tempfile.mkdtemp()
    try:
        tempname = os.path.join(tempdir, 'new.zip')
        with ZipFile(file_name, 'r') as r, ZipFile(tempname, 'w') as w:
            for item in r.infolist():
                data = r.read(item.filename)          
                data = change(data)
                w.writestr(item, data)
        shutil.move(tempname, file_name)
    finally:
        shutil.rmtree(tempdir)


def get_excel_files():
    if not os.path.isdir(EXCELS_DIR):
        os.mkdir('results')
        print("Files being downloaded to ", EXCELS_DIR)
        parse_url_for_excels(URL_RESULTS)
        fix_xlsx_errors()
    else:
        print(f"Les fichiers sont déjà dans le répertoire \"{EXCELS_DIR}/\" et déjà traités, aucun nouveau fichier téléchargé ni traité.")


**L'appel à la commande *get_excel_files.xlsx* ne fait rien si le répertoire *results/* existe. Il est crée lors du 1er téléchargement** 

In [3]:
get_excel_files()

Les fichiers sont déjà dans le répertoire "results/" et déjà traités, aucun nouveau fichier téléchargé ni traité.


In [4]:
%%time
def create_dataframe_from_files(path, verbose=True):
    """
    Sélection des fichiers et concatenate dans une dataframe
    
    Returns a dataframe
    """
    if not verbose:
        print("Quiet mode activated. Be patient...", end='')
    dfs=[]
    for filename in [f for f in glob.glob(path + "/*.xlsx")]:
        # on exclu les fichiers donnant des infos sur les concurrents arrivées (à partir du 27 janvier)
        # on exclu le 1er fichier au départ qui est sans données
        if filename >= path+'/vendeeglobe_20210127_170000.xlsx' or filename == path+"/vendeeglobe_20201108_120200.xlsx":
            continue
        if verbose:
            print('Including file ', filename)

        # read excel
        x = pd.read_excel(filename, 
                          dtype=object,
                          skiprows=[1, 2, 3], 
                          header=1, 
                          usecols=range(1,21), 
                          skipfooter=4)
        # do not use col 0
        # remove footer
        # rename 1  Classement
        # split 2 on \n rename Pays /  Voile
        # split 3 on \n ren Skipper /  Bateau
        # change names

        # ajout colonne avec le timestamp du fichier d'où est extrait la data
        x['Fichier de resultats'] = filename[-20:-5]

        dfs.append(x)
    
    df = pd.concat(dfs , ignore_index=True)
    #df.duplicated().value_counts()

    if not verbose:
        print("done")
    return df

def clean_data(df, verbose=True):
    """
    Cleanup des noms de colonnes
    """
    print('Cleaning dataframe... ', end='')

    # cleanup sur nom dde cols, etc.
    df.rename(inplace=True, columns={'Unnamed: 1': 'Classement', 'Unnamed: 2': 'Pays', 'Unnamed: 3': 'Skipper', 
                                    'Unnamed: 19': 'DTF', 'Unnamed: 20': 'DTL', "Heure FR\nHour FR": "Heure FR"})
    df[['Pays','Voile']] = df["Pays"].str.extract("(.*)\n(.*)").astype(str)
    df[['Skipper','Bateau']] = df["Skipper"].str.extract("(.*)\n(.*)").astype(str)
    df[['Heure FR']] = df["Heure FR"].str.extract("(.*)\n.*").astype(str)
    df.rename(inplace=True, columns={'Latitude\nLatitude': 'Latitude', 
                    'Longitude\nLongitude': 'Longitude', 
                    'Cap\nHeading': 'Cap 30m', 'Vitesse\nSpeed': 'Vitesse 30m', 
                    'VMG\nVMG': 'VMG 30m', 'Distance\nDistance': 'Distance 30m',
                    'Cap\nHeading.1': 'Cap dernier', 'Vitesse\nSpeed.1': 'Vitesse dernier', 
                    'VMG\nVMG.1': 'VMG dernier', 'Distance\nDistance.1': 'Distance dernier',
                    'Cap\nHeading.2': 'Cap 24h', 'Vitesse\nSpeed.2': 'Vitesse 24h', 
                    'VMG\nVMG.2': 'VMG 24h', 'Distance\nDistance.2': 'Distance 24h'})

    for col in ['Vitesse 30m','VMG 30m', 'Vitesse dernier', 'VMG dernier', 'Vitesse 24h', 'VMG 24h']:
        df[[col]] = df[col].str.extract("(.*) kts").astype(float)
    for col in ['Distance 30m', 'Distance dernier', 'Distance 24h', 'DTF', 'DTL']:
        df[[col]] = df[col].str.extract("(.*) nm").astype(float)
    for col in ['Cap 30m', 'Cap dernier', 'Cap 24h']:
        df[[col]] = df[col].str.extract("(\d*)°").astype(float)

    # classement des abandons trasnformé en int
    df[['Classement']] = df[['Classement']].applymap(lambda x: x.replace('RET', '-1')).astype(str)
    df[['Classement']] = df[['Classement']].applymap(lambda x: x.replace('NL', '-2')).astype(str)
    
    
    # suppression des lignes avec Nan
    # ce sont les abandons
    df = df.dropna()

    print("done")
    return df


# backup dans un fichier pickle pour ne pas retraiter à chaque run
if not os.path.isfile(PICKLE_DF):
    print(f"\nLoading data from {EXCELS_DIR}/*.xlsx files")
    df = create_dataframe_from_files(EXCELS_DIR, verbose=False)
    df = clean_data(df)
    df.to_pickle(PICKLE_DF)
    print(f"{PICKLE_DF} saved")
else:
    print(f"\nLoading data from pickle file {PICKLE_DF}...", end='')
    df = pd.read_pickle(PICKLE_DF)
    print(' done')
    print(f'\n(Note: if you need to reset pickle content remove file {PICKLE_DF} manually)')
    
df.shape



Loading data from pickle file pickle_classement.pkl... done

(Note: if you need to reset pickle content remove file pickle_classement.pkl manually)
CPU times: user 17.6 ms, sys: 4.11 ms, total: 21.7 ms
Wall time: 20.4 ms


(13703, 23)

In [5]:
df.iloc[10000]

Classement                                     29
Pays                                             
Skipper                            Clément Giraud
Heure FR                                 12:30 FR
Latitude                               38°56.43'N
Longitude                              19°07.68'W
Cap 30m                                     228.0
Vitesse 30m                                   4.2
VMG 30m                                       3.3
Distance 30m                                  2.1
Cap dernier                                 177.0
Vitesse dernier                               3.8
VMG dernier                                   3.7
Distance dernier                             11.4
Cap 24h                                     241.0
Vitesse 24h                                   8.2
VMG 24h                                       5.7
Distance 24h                                196.7
DTF                                       23557.6
DTL                                         370.4




#### Extraction des caractéristiques techniques de chacun des voiliers.

* Extraction des caractéristiques techniques de chacun des voiliers depuis la page glossaire 
* ajout des informations sur la présence de foils depuis la page classement



Traitement manuel de la page web https://www.vendeeglobe.org/fr/classement pour récupérer l'info sur les foils et le classement final

In [6]:
def get_infos_from_classement(df):
    """
    Récupère les infos skipper, foil, etc.
    
    Retourne une liste de liste de [classement final, nom skipper, Oui/Non (foil)]
    """
    print("Getting boats infos...")
    classement_html=""
    if not os.path.isfile(FILE_RESULTS):
        print("Reading file from far away")
        req = requests.get(URL_RESULTS)
        classement_html = req.content
        open(os.path.join(FILE_RESULTS), 'wb').write(classement_html)
    else:
        print("Reading file locally")
        with open(FILE_RESULTS,'r') as file:
            classement_html = file.read()

    soup = get_soup_from_file(classement_html)
    skippers_info=[]
    for ranking_row in soup.find_all("tr", {"class": "ranking-row"}):
        cell_rank = ranking_row.find('td', attrs={'class': 'row-number'} ).text
        cell_skipper = ranking_row.find('td', attrs={'class': 'row-skipper'} ).contents[2]
        cell_skipper = re.search(r'\n\s+(\w[\s\'\w-]*)', cell_skipper).group(1).title()
        cell_has_foil = ranking_row.find('td', attrs={'class': 'row-layout'} ).text
        skippers_info.append([cell_rank, cell_skipper, cell_has_foil])
    return skippers_info

# Extraction des caractéristiques techniques de chacun des voiliers.
def get_infos_from_glossaire():
    """
    Récupère les infos skipper, foil, etc.
    """
    if not os.path.isfile(FILE_GLOSSAIRE):
        print("Reading file from far away..." , end='')
        req = requests.get(URL_GLOSSAIRE)
        glossaire_html = req.content
        open(os.path.join(FILE_GLOSSAIRE), 'wb').write(glossaire_html.content)
    else:
        print("Reading file locally... " , end='')
        with open(FILE_GLOSSAIRE,'r') as file:
            glossaire_html = file.read()

    soup = get_soup_from_file(glossaire_html)
    
    tech_info={}
    
    boats_popup_infos = soup.find_all('div', attrs={'class': 'boats-list__popup-infos'})
    specs_list = soup.find_all('ul', attrs={'class': 'boats-list__popup-specs-list'})
#     print(len(boats_popup_infos), len(specs_list))
#     v = []
    for i in range(len(boats_popup_infos)):
        bateau = boats_popup_infos[i].h3.text
        specs = specs_list[i]
        voile = specs.find(string=re.compile("Numéro de voile : "))
        if voile == None:
            voile = 0
        else:
            voile = re.match(".*: ([\w\s]+)", voile)[1]
        anc_name = specs.find(string=re.compile("Anciens noms du bateau : "))
        if anc_name==None:
            anc_name = bateau
        else:    
            anc_name = re.match(".*: ([,\w\s]+)", anc_name)[1]
        Architecte = specs.find(string=re.compile("Architecte")) # : Marc Lombard</li>
        Architecte = re.match(".*: ([\s\w]+)", Architecte)[1]
        Chantier= specs.find(string=re.compile("Chantier")) #MAG France</li>
        Chantier= re.match(".*: ([\s\w]+)", Chantier)[1]
        lancement= specs.find(string=re.compile("Date de lancement")) # : 01 Mars 1998</li>
        lancement= re.match(".*: ([\s\w]+)", lancement)[1]
        Longueur= specs.find(string=re.compile("Longueur")) # : 18,28m</li>
        Longueur= re.match(".*: ([,.\d]+)", Longueur)[1]
        Largeur = specs.find(string=re.compile("Largeur")) # : 5,54m</li>
        Largeur = re.match(".*: ([,\d]+)", Largeur)[1]
        Tirant = specs.find(string=re.compile("Tirant d'eau")) # : 4,50m</li>
        Tirant = re.match(".*: ([,\d]+)", Tirant)[1]
        poids = specs.find(string=re.compile("Déplacement")) # : 9t</li>
        poids = re.match(".*: ([,\dncNC]+)\s?t?", poids)[1]
        if poids=="nc" or poids=="NC":
            poids="0"
        derives = specs.find(string=re.compile("Nombre de dérives")) # : 2</li>
        derives = re.match(".*: (.*)", derives)[1]
        mat = specs.find(string=re.compile("Hauteur mât")) # : 29 m</li>
        mat = re.match(".*: ([,\d]+)", mat)[1]
        quille = specs.find(string=re.compile("Voile quille")) # : acier</li>
        if quille == None:
            quille = "NC"
        else:
            quille = re.match(".*: ([\s\w]+)", quille)[1] 
        Surface_pres = specs.find(string=re.compile("Surface de voiles au près")) # : 260 m2</li>
        Surface_pres = re.match(".*: ([,\d]+).*m[2²]", Surface_pres)[1]
        Surface_portant = specs.find(string=re.compile("Surface de voiles au portant")) # : 580 m2</li>
        Surface_portant = re.match(".*: ([,\d]+).*m[2²]", Surface_portant)[1]

        # manual cleanup des numéros de voiles qui ne matchent pas avec les fichiers classements
        if bateau == 'LinkedOut':
            voile = "FRA 59"
        if voile == "001":
            voile = "FRA 01"
        if voile == "4":
            voile = "FRA 4" 
        if voile == "2":
            voile = "FRA 02"
#         if voile == "6":
#                 voile = "FRA 6"
        if voile == "08":
            voile = "FRA 8"
        if voile == "16":
            voile = "MON 10"
        if voile == "17":
            voile = "FRA 17"
        if voile == "18":
            voile = "FRA 18"
        if voile == "69":
            voile = "FRA 69"
        if voile == "SUI07":
            voile = "SUI 7"
        if voile == "GBR77":
            voile = "GBR 777"
        if voile[3] != " ":
#             print(voile)
            voile = voile[0:3]+" "+voile[3:]
#             print(voile)
        tech_info[voile] = {'Voile': voile, 'Nom bateau': bateau, 'Longeur': Longueur, 
                            'Largeur': Largeur, 'Tirant': Tirant, 
                           'Poids': poids, "Dérives": derives,
                           'Hauteur mât': mat, "Quille": quille, 
                            "Surface près": Surface_pres,  "Surface portant": Surface_portant,
                           "Année lancement": lancement[-4:], 'Ancien nom': anc_name}
#         v.append(voile)
    print("Done")
    return tech_info



In [7]:

foils_etc = get_infos_from_classement(df)

# manual fixes
# skippers with names not matching
# from web page 2 errors
skips=set()
for f in foils_etc:
    skipper=f[1]
    skips.add(skipper)
all_skips = set(df['Skipper'])
print(skips.difference(set(all_skips)))
df[['Skipper']] = df[['Skipper']].applymap(lambda x: x.replace('Arnaud Boissieres', 'Arnaud Boissières')).astype(str)

for f in foils_etc:
    if f[1]=="Sam Davies":
        skipper = "Samantha Davies"
    elif f[1]=="Alan  Roura":
        skipper="Alan Roura"
    else:
        skipper=f[1]
    foil=f[2]
    
    df.loc[df['Skipper']==skipper, 'Foil'] = foil
    df.loc[df['Skipper']==skipper, 'Classement final'] = f[0]

df[['Classement final']] = df[['Classement final']].applymap(lambda x: x.replace('ABD', '-1')).astype(str)
df[['Classement final']] = df[['Classement final']].astype(int)
df[['Classement']] = df[['Classement']].astype(int)


# set(df['Skipper'])
df[['Foil']] = df[['Foil']].applymap(lambda x: x.replace('Oui', '1')).astype(str)
df[['Foil']] = df[['Foil']].applymap(lambda x: x.replace('Non', '0')).astype(str)
df[['Foil']] = df[['Foil']].astype(int)
df[['Foil']] = df[['Foil']].astype(bool)



Getting boats infos...
Reading file locally
{'Sam Davies', 'Arnaud Boissières', 'Alan  Roura'}


In [49]:
#################################################################################
# MERGE DES CLASSEMENTS ET DONNEES TECHNIQUES


# tech_info contains a dict of technical details
# backup dans un fichier pickle pour ne pas retraiter à chaque run
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#brief-primer-on-merge-methods-relational-algebra

def merge(df):
    if not os.path.isfile(PICKLE_DF_TECH):  
        print(f"\nLoading boats technical data")
        tech_info = get_infos_from_glossaire()

        print(f"\nMerging df with technical data")
        df_tech_info = pd.DataFrame(tech_info).T
        df_merge = pd.merge(df, df_tech_info, on="Voile")

        print(f"Saving pickle file {PICKLE_DF_TECH}")
        df_merge.to_pickle(PICKLE_DF_TECH)
    else:
        print(f"\nLoading data from pickle file {PICKLE_DF_TECH}...", end='')
        df_merge = pd.read_pickle(PICKLE_DF_TECH)
        print(' done')
        print(f'\n(Note: if you need to renew pickle content remove file {PICKLE_DF_TECH} manually)')
    return df_merge

df_merge = merge(df)



Loading data from pickle file pickle_classement_et_tech.pkl... done

(Note: if you need to renew pickle content remove file pickle_classement_et_tech.pkl manually)


In [50]:
## cleanup columns types, renaming and ordering

df_merge['Tirant'] = df_merge["Tirant"].str.replace(',','.').astype(float)
df_merge['Longeur'] = df_merge["Longeur"].str.replace(',','.').astype(float)
df_merge['Largeur'] = df_merge["Largeur"].str.replace(',','.').astype(float)
df_merge['Poids'] = df_merge["Poids"].str.replace(',','.').astype(float)
df_merge["Hauteur mât"] = df_merge["Hauteur mât"].str.replace(',','.').astype(float)
df_merge["Surface près"] = df_merge["Surface près"].str.replace(',','.').astype(float)
df_merge["Surface portant"] = df_merge["Surface portant"].str.replace(',','.').astype(float)
df_merge["Année lancement"] = df_merge["Année lancement"].astype(int)

columns=['Classement', 'Classement final', 'Skipper', 'Bateau', 'Voile', 'Pays', 'Heure FR', 'Latitude', 'Longitude',
       'Cap 30m', 'Vitesse 30m', 'VMG 30m', 'Distance 30m', 'Cap dernier',
       'Vitesse dernier', 'VMG dernier', 'Distance dernier', 'Cap 24h',
       'Vitesse 24h', 'VMG 24h', 'Distance 24h', 'DTF', 'DTL',
       'Fichier de resultats', 'Foil', 'Dérives', 'Tirant', 'Longeur', 'Largeur', 'Poids', 
       'Hauteur mât', 'Quille', 'Surface près', 'Surface portant',
       'Année lancement', 'Ancien nom']

df_merge = df_merge[columns]

In [52]:
# check for Nan values
# df.loc[df.isna().any(axis=1)]

df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13703 entries, 0 to 13702
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Classement            13703 non-null  int64  
 1   Classement final      13703 non-null  int64  
 2   Skipper               13703 non-null  object 
 3   Bateau                13703 non-null  object 
 4   Voile                 13703 non-null  object 
 5   Pays                  13703 non-null  object 
 6   Heure FR              13703 non-null  object 
 7   Latitude              13703 non-null  object 
 8   Longitude             13703 non-null  object 
 9   Cap 30m               13703 non-null  float64
 10  Vitesse 30m           13703 non-null  float64
 11  VMG 30m               13703 non-null  float64
 12  Distance 30m          13703 non-null  float64
 13  Cap dernier           13703 non-null  float64
 14  Vitesse dernier       13703 non-null  float64
 15  VMG dernier        

In [53]:
df_merge.iloc[10000]

Classement                                  18
Classement final                            11
Skipper                           Armel Tripon
Bateau                  L'Occitane en Provence
Voile                                   FRA 02
Pays                                        FR
Heure FR                              15:00 FR
Latitude                            46°18.29'N
Longitude                           08°58.37'W
Cap 30m                                  194.0
Vitesse 30m                                9.8
VMG 30m                                    9.7
Distance 30m                               4.9
Cap dernier                              195.0
Vitesse dernier                           10.9
VMG dernier                               10.9
Distance dernier                          32.8
Cap 24h                                  266.0
Vitesse 24h                               12.4
VMG 24h                                    7.1
Distance 24h                             298.4
DTF          

In [11]:
### back to df 

df = df_merge

############################## AT WORK BELOW THIS LINE #####################################


* Extraction des caractéristiques techniques de chacun des voiliers.
* Rapprochement des données des voiliers avec celle des classements.
* Corrélation et régression linéaire entre le classement (rang) et la vitesse utile (VMG) des voiliers.
* Impact de la présence d'un foil sur le classement et la vitesse des voiliers.
* Visualisation de la distance parcourue par voilier.
* Cartes avec les routes d'un ou plusieurs voiliers.
* Analyses de séries temporelles.
* Application d'algorithmes statistiques ou de machine learning.
* etc.